In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sandeephulsur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sandeephulsur/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


Using TensorFlow backend.


In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    "let's start over"],
   'responses': ["Hello, I'm a COVID-19 HealthBot.\nType Menu to see the services we provide"],
   'context': ['']},
  {'tag': 'Menu',
   'patterns': ['Menu',
    'menu',
    'what is the menu',
    'What services do you provide',
    'services'],
   'responses': ['COVID-19 Information \n Find Hospitals \n Type the service you want to access'],
   'context': ['']},
  {'tag': 'Covid_info',
   'patterns': ['covid-19 information',
    'covid info',
    'corona information',
    'corona info'],
   'responses': ['Great! To ask a question, type it into the box below.'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
  

In [4]:

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

81 documents
20 classes ['Coronavirus', 'Covid_info', 'Menu', 'SARS', 'adverse_drug', 'animals', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'highrisk_covid', 'hospital_search', 'hospitals', 'options', 'pharmacy_search', 'protect', 'spread', 'symptoms', 'thanks', 'vaccine']
137 unique lemmatized words ["'s", '(', ')', ',', '14', 'a', 'adverse', 'all', 'and', 'andprotection', 'animal', 'anyone', 'are', 'area', 'at', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'catch', 'causing', 'chatting', 'check', 'corona', 'coronavirus', 'could', 'covid', 'covid-19', 'data', 'day', 'detail', 'disease', 'do', 'doe', 'dont', 'drug', 'entry', 'find', 'for', 'from', 'get', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'high', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'in', 'info', 'information', 'is', 'it', 'later', 'let', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'measure', 'menu', 'module

In [6]:

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))



In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1


    training.append([bag, output_row])

In [8]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


In [9]:

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
81/81 [==============================] - 0s 2ms/step - loss: 2.9691 - accuracy: 0.1111
Epoch 2/200
81/81 [==============================] - 0s 205us/step - loss: 2.9210 - accuracy: 0.1358
Epoch 3/200
81/81 [==============================] - 0s 200us/step - loss: 2.8182 - accuracy: 0.1481
Epoch 4/200
81/81 [==============================] - 0s 199us/step - loss: 2.7259 - accuracy: 0.2222
Epoch 5/200
81/81 [==============================] - 0s 253us/step - loss: 2.6062 - accuracy: 0.2840
Epoch 6/200
81/81 [==============================] - 0s 235us/step - loss: 2.4636 - accuracy: 0.3086
Epoch 7/200
81/81 [==============================] - 0s 215us/step - loss: 2.1732 - accuracy: 0.4198
Epoch 8/200
81/81 [==============================] - 0s 206us/step - loss: 2.0031 - accuracy: 0.4444
Epoch 9/200
81/81 [==============================] - 0s 183us/step - loss: 1.9727 - accuracy: 0.4444
Epoch 10/200
81/81 [==============================] - 0s 189us/step - loss: 1.8578 - accuracy